In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import pandas_bokeh
# import finplot as fplt

import polygon_rest_api as papi
import polygon_backfill as pb
# import ray_backfill as rb
import polygon_s3 as ps3
import bar_samples as bs
import bar_labels as bl

# import filters as ft
# import market_cluster as mc

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20, 7]
from matplotlib import style
style.use('fivethirtyeight')

# import ipdb
# ipdb.set_trace(context=10)

# df.style.background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG' are other good diverging colormaps
# pd.set_option('precision', 2)

# get local market daily data
# mdf = pd.read_parquet('data/mdf.parquet')

# .plot_bokeh(sizing_mode="scale_height")

In [8]:
symbol='GLD'
# symbol = 'market'
date = '2020-01-02'
start_date = '2020-01-01' 
end_date = '2020-01-10'
symbols=['SPY', 'QQQ']
tick_type='trades'
# tick_type = None

result_path = '/Users/bobcolner/QuantClarity/pandas-polygon/data'

In [9]:
open_dates = pb.get_open_market_dates(start_date, end_date)

In [ ]:
def fill_gap_renko(last_bar, first_bar, renko_size: float):
    num_steps = round(abs(last_bar['price_wmean'] - first_bar['price_wmean']) / renko_size) + 2
    fill_prices = np.linspace(start=last_bar['price_wmean'], stop=first_bar['price_wmean'], num=num_steps)
    fill_dt = pd.date_range(start=last_bar['close_at'], end=first_bar['open_at'], periods=num_steps)
    fill_dict = {'close_at': fill_dt, 'price_wmean': fill_prices, 'price_high': fill_prices+(renko_size/2), 'price_low': fill_prices-(renko_size/2)}
    return fill_dict


thresh = {
    # GT thresholds
    'renko_size': 0.2,
    'renko_reveral_multiple': 2,
    'duration_sec': 60*30,
    'tick_imbalance': 150,
    # LT thresholds 
    'min_duration_sec': 15,
    'min_tick_count': 30,
}

bar_dates = []
bar_dates_d = {}
n = 0
for date in open_dates:
    n += 1
    # load ticks
    tdf = ps3.load_ticks(result_path, symbol, date)
    # sample bars
    bars, state = bs.build_bars(tdf, thresh)
    
    bar_dates.append(bars)
    bar_dates_d[date] = bars

In [201]:
pd.DataFrame(bar_dates_d[date])

,bar_trigger,open_at,close_at,duration_td,duration_sec,duration_min,price_open,price_close,price_low,price_high,...,price_wstd,tick_count,volume_sum,dollar_sum,tick_imbalance,volume_imbalance,dollar_imbalance,tick_imbalance_run,volume_imbalance_run,dollar_imbalance_run
0,duration,2020-01-10 09:03:37.772587461,2020-01-10 13:00:37.725308837,0 days 03:56:59.952721376,14219,236,146.009995,145.869995,145.750000,146.009995,...,0.027320,30,25016,3.649297e+06,1,21640,3.156694e+06,5,15690,2.288700e+06
1,renko_up,2020-01-10 13:01:01.215090956,2020-01-10 13:30:31.984623219,0 days 00:29:30.769532263,1770,29,145.840103,146.289993,145.679993,146.289993,...,0.090282,49,10629,1.549142e+06,-14,-8073,-1.176308e+06,4,223,3.259585e+04
2,renko_up,2020-01-10 13:30:31.984632807,2020-01-10 13:34:22.230912043,0 days 00:03:50.246279236,230,3,146.289993,146.500000,146.250000,146.500000,...,0.052829,32,7357,1.076502e+06,7,-443,-6.455342e+04,5,725,1.061727e+05
3,renko_down,2020-01-10 13:34:28.594230778,2020-01-10 13:50:06.917771214,0 days 00:15:38.323540436,938,15,146.490005,146.089996,146.089996,146.550003,...,0.125524,136,37142,5.437744e+06,-1,4172,6.122591e+05,16,3341,4.882048e+05
4,renko_down,2020-01-10 13:50:06.917778793,2020-01-10 13:52:26.881887161,0 days 00:02:19.964108368,139,2,146.089996,145.800003,145.800003,146.089996,...,0.075249,30,6779,9.888070e+05,-25,-6290,-9.173722e+05,19,6015,8.772371e+05
5,duration,2020-01-10 13:52:31.098443230,2020-01-10 14:22:37.077753622,0 days 00:30:05.979310392,1805,30,145.830002,146.179993,145.699997,146.179993,...,0.135140,142,42044,6.135218e+06,20,12912,1.885295e+06,6,1357,1.983162e+05
6,tick_imbalance,2020-01-10 14:22:37.077773140,2020-01-10 14:32:06.310319544,0 days 00:09:29.232546404,569,9,146.179993,146.270004,146.160004,146.289993,...,0.026520,491,195446,2.858035e+07,150,119229,1.743595e+07,18,3300,4.826910e+05
7,tick_imbalance,2020-01-10 14:32:06.310337736,2020-01-10 14:56:31.299025422,0 days 00:24:24.988687686,1464,24,146.270004,146.289993,146.190002,146.375000,...,0.042917,2556,502289,7.347246e+07,-150,20821,3.050172e+06,16,3800,5.559020e+05
8,tick_imbalance,2020-01-10 14:56:31.299228213,2020-01-10 15:08:16.649586966,0 days 00:11:45.350358753,705,11,146.289993,146.309998,146.220001,146.360001,...,0.033340,781,173370,2.536223e+07,-150,-2948,-4.309103e+05,4,475,6.949725e+04
9,renko_up,2020-01-10 15:08:17.237060400,2020-01-10 15:25:01.428158157,0 days 00:16:44.191097757,1004,16,146.309998,146.518097,146.227402,146.518097,...,0.059163,1345,237812,3.480508e+07,17,1016,1.502395e+05,0,0,0.000000e+00


In [157]:
last_bar = bar_dates[0][-1]
first_bar = bar_dates[1][0]

fill_dict = fill_gap_renko(last_bar, first_bar, renko_size)

In [163]:
bar_dates[0].append(fill_dict)

In [ ]:
# sample ticks -> bars
renko_size = 0.2
# renko_reversal = 2
thresh = {
    # GT thresholds
    'renko_size': renko_size,
    'renko_reveral_multiple': 2,
    'duration_sec': 60*30,
    'tick_imbalance': 150,
    # LT thresholds 
    'min_duration_sec': 15,
    'min_tick_count': 30,
}
# ticks_df['epoch'] = ticks_df.date_time.astype(int)
tdf = ticks_df.loc[(ticks_df.date_time.dt.hour >= 13) & (ticks_df.date_time.dt.hour <= 22)]
bars, _ = bs.build_bars(ticks_df=tdf, thresh=thresh)
len(bars)

bars_df = pd.DataFrame(bars).set_index('close_at')
# bars_df = pd.DataFrame(bars)

In [ ]:
# range / price = HFT profit potential
(df.price.max() - df.price.min()) / df.price.mean()

In [ ]:
# label bars
label_horizon_mins = 30
risk_level = 0.2
# reward_base = 1
reward_ratios = np.arange(3, 20, .5)

lbars = bl.label_bars(bars, ticks_df, risk_level, label_horizon_mins, reward_ratios)

lbars_df = pd.DataFrame(lbars)

lbars_unq = bl.get_concurrent_stats(lbars_df)
print(lbars_unq['grand_avg_unq'])

In [ ]:
lbars_df[['price_wmean', 'label_rrr']].plot_bokeh(sizing_mode="scale_height")
# 17, 31

In [ ]:
lbars_df[['price_wmean', 'label_rrr']].plot(subplots=True, figsize=[17,12])

In [ ]:
label_prices, price_end_at = bl.get_label_ticks(tdf, bars[31]['close_at'], 30)